# Data Import

In [ ]:
import pandas as pd
df = pd.read_parquet("data/args.parquet")

# Embeddings

In [ ]:
import pandas as pd
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
df["conclusion_embedded"] = df["conclusion"].apply(lambda x: embed([x]).numpy()[0])

In [ ]:
del(embed)

In [ ]:
df.to_parquet("data/embeddings.parquet")

# Clustering

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_parquet("data/embeddings.parquet")

In [ ]:
ID = np.vstack(df["id"].values)
X = np.vstack(df["conclusion_embedded"].values)

In [ ]:
from cuml.cluster import KMeans

clusterer = KMeans(n_clusters=300)
clusterer.fit(X)
clusters = clusterer.labels_

In [ ]:
df = pd.DataFrame(zip(ID,X,clusterer.labels_), columns=["id","embedding","cluster"])

In [ ]:
centroids = (
    df
    .loc[:,["embedding","cluster"]]
    .groupby("cluster")
    .apply(lambda group: np.vstack(group.values).mean(axis=0)[0])
    .reset_index()
    .rename({"cluster":"cluster", 0:"centroid"}, axis=1)
)

In [ ]:
df["id"] = df["id"].apply(lambda x: x[0])
df = df.merge(centroids, on="cluster", how="left").drop("embedding",axis=1)

In [ ]:
df.to_parquet("data/clusters.parquet")